# РК2


## 31 вариант

**Задание.** Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

**Задание по варианту:** метод опорных векторов, градиентный бустинг.

**Датасет:** https://www.kaggle.com/datasets/kmldas/loan-default-prediction (файл Default_Fin.csv)

В качестве набора данных предлагается датасет с бразильскими домами для аренды.

В данном датасете 5 колонок:

* **index** - индификатор;
* **Employed** - тродоустройства; 
* **Bank Balance** - баланс в банке;
* **Annual Salary** - годовая зароботная плата;
* **Defaulted** - дефолт;

In [42]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from io import StringIO
from IPython.display import Image
import graphviz
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
from sklearn.metrics import roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")
from sklearn.preprocessing import StandardScaler

## Загрузка и первичный анализ данных


In [43]:
data = pd.read_csv('C:\\Users\\adolf\\Downloads\\Default_Fin.csv', sep=",")

In [44]:
data.shape

(10000, 5)

In [45]:
# типы колонок
data.dtypes

Index              int64
Employed           int64
Bank Balance     float64
Annual Salary    float64
Defaulted?         int64
dtype: object

In [46]:
data.head(5)

,Index,Employed,Bank Balance,Annual Salary,Defaulted?
0,1,1,8754.36,532339.56,0
1,2,0,9806.16,145273.56,0
2,3,1,12882.60,381205.68,0
3,4,1,6351.00,428453.88,0
4,5,1,9427.92,461562.00,0


In [47]:
data.rename(columns={'Defaulted?': "Defaulted"}, inplace=True)

In [48]:
# кол-во пропущенные значений (тк нулевые значения в данном датасете - конкретное значения)
nan_count = (data.isna().sum())
print(nan_count)

Index            0
Employed         0
Bank Balance     0
Annual Salary    0
Defaulted        0
dtype: int64


## Обрабработка данных

## Разделение на обучающую и тестовую выборки.

In [49]:
target = data['Defaulted']
data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(
    data, target, test_size=0.2, random_state=1)

In [50]:
data_X_train.shape, data_X_test.shape, data_y_train.shape, data_y_test.shape

((8000, 5), (2000, 5), (8000,), (2000,))

# Метод опорных векторов

In [51]:
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR

svr_1 = SVR() # задача регрессии
svr_1.fit(data_X_train, data_y_train)

SVR()

In [52]:
data_y_pred_1 = svr_1.predict(data_X_test)

In [53]:
mean_absolute_error(data_y_test, data_y_pred_1), mean_squared_error(data_y_test, data_y_pred_1)

(0.12183517774277966, 0.033251900283615854)

## Модель с масштабированием данных

In [54]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

In [55]:
target_2 = data_scaled['Defaulted']
data_X_train_2, data_X_test_2, data_y_train_2, data_y_test_2 = train_test_split(
    data_scaled, target_2, test_size=0.2, random_state=1)

In [56]:
svr_2 = SVR() # задача регрессии
svr_2.fit(data_X_train_2, data_y_train_2)

SVR()

In [57]:
data_y_pred_2 = svr_2.predict(data_X_test_2)

In [58]:
mean_absolute_error(data_y_test_2, data_y_pred_2), mean_squared_error(data_y_test_2, data_y_pred_2)

(0.027030693519331128, 0.0011603369274479803)

# Градиентный бустинг

## Модель градиентного бустинга с использованием библиотеки catboost для бинарной классификации по признаку "Employed"

In [59]:
!pip install catboost

In [61]:
from catboost import CatBoostClassifier

target_3 = data['Employed']
data_X_train_3, data_X_test_3, data_y_train_3, data_y_test_3 = train_test_split(
    data, target_3, test_size=0.2, random_state=1)

ct = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, loss_function='Logloss')

ct.fit(data_X_train_3, data_y_train_3, verbose=False)

data_y_pred_3 = ct.predict(data_X_test_3)

In [62]:
mean_absolute_error(data_y_test_3, data_y_pred_3), mean_squared_error(data_y_test_3, data_y_pred_3)

(0.0, 0.0)

In [63]:
accuracy_score(data_y_test_3, data_y_pred_3)

1.0

In [64]:
f1_score(data_y_test_3, data_y_pred_3, average='micro')

1.0

In [65]:

f1_score(data_y_test_3, data_y_pred_3, average='macro')

1.0

In [66]:
f1_score(data_y_test_3, data_y_pred_3, average='weighted')

1.0

**Вывод**: модель градиентного бустинга показала себя лучше, чем модель, основанная на методе опорных векторов